# Geo Overview

Jupyter notebooks provide a natural home for embedded, interactive maps although we can also use embdded code to generate a wide range of static maps too.

A wide range of Python packages exist that support the creation and embedding of interactive maps built from third party mapping services such as OpenStreetMap (OSM).

Packages can also work with tiles served from a personally hosted map tile server.

## `ipyleaflet`

The [`ipyleaflet`](https://ipyleaflet.readthedocs.io/en/latest/) package supports the creation of embedded, interactive maps as an `ipywidget` using the [`leaflet.js`](https://leafletjs.com/) Javascript package.

In [1]:
%%capture
try:
    import ipyleaflet
except:
    %pip install ipyleaflet

In [2]:
import ipyleaflet as lflt

m = lflt.Map(center=[52.0250, -0.7084], zoom=14)
m

Map(center=[52.025, -0.7084], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

## `folium`

The [`folium`](https://python-visualization.github.io/folium/) package supports the creation and embedding of interactive maps using `leaflet.js` although in this case the embedded map is *not* wrapped as an `ipywidget`.

In [3]:
%%capture
try:
    import folium
except:
    %pip install folium

In [4]:
import folium

m = folium.Map(location=[52.0250, -0.7084], zoom_start=14)
m

### Adding Markers

Adding one or more markers to a map is relatively straightforward:

In [5]:
m = folium.Map(location=[52.0250, -0.7084], zoom_start=14, tiles='Stamen Toner')

folium.Marker([52.0250, -0.7084],
              popup='Open University, Walton Hall').add_to(m)
m

## Adding Boundaries

Boundaries represented using a `.geojson` file can be rendered as an overlayed map layer.

In [6]:
import os
m = folium.Map( location=[52.0250, -0.7084], zoom_start=9 )

#Path to geojson file
mk = os.path.join('boundaries', 'mk.json')

#Add geojson layer to map
folium.GeoJson( mk, name='geojson' ).add_to(m)

m

### Choropleth Maps

We can easily create a choropleth map from a tabular dataset that shares keys with a shapefile.

In [7]:
%%capture
try:
    import pandas
except:
    %pip install pandas

In [8]:
import pandas as pd

turnout=pd.read_csv('data/iw_turnout.csv')
turnout.head()

,Electoral Division,Candidate,Electorate,NoOnRoll,turnout
0,Arreton and Newchurch,Total votes cast,1286,3070,0.418893
1,Binstead and Fishbourne,Total votes cast,1160,2766,0.419378
2,"Brading, St Helens and Bembridge",Total votes cast,4640,6189,0.749717
3,Carisbrooke,Total votes cast,883,2632,0.335486
4,Central Wight,Total votes cast,1339,2904,0.461088


Various tools exist that allow us to read and process geojson files in a convenient way.

For example, use `fiona` to read in a geojson file, parse the extent of the shape file and find the mid-point:

In [9]:
%%capture
try:
    import fiona
except:
    %pip install fiona

In [10]:
import fiona

with fiona.open('boundaries/iw.json') as fi:
    latlong = [(fi.bounds[1]+fi.bounds[3])/2,
               (fi.bounds[0]+fi.bounds[2])/2]

Center the map on that mid-point and add a choropleth layer:

In [11]:
m = folium.Map(location=latlong )

folium.Choropleth(
   geo_data='boundaries/iw.json',
    data=turnout,
    columns=['Electoral Division', 'turnout'],
    key_on='feature.properties.WD13NM',
    fill_color='PuBuGn', fill_opacity=0.7
).add_to(m)

m